In [18]:
import numpy as np
import pandas as pd

# reproducibility
np.random.seed(42)

# === Оси времени ===
years = [2020, 2021, 2022, 2023, 2024]
months = ["Январ","Феврал","Март","Апрел","Май","Июн",
          "Июл","Август","Сентябр","Октябр","Ноябр","Декабр"]

# === Сезонные сдвиги климата (как у тебя было) ===
month_temp_shift = {"Январ":-10,"Феврал":-8,"Март":-4,"Апрел":2,"Май":8,
                    "Июн":12,"Июл":14,"Август":12,"Сентябр":8,
                    "Октябр":2,"Ноябр":-4,"Декабр":-8}
month_rain_shift = {"Январ":30,"Феврал":35,"Март":60,"Апрел":80,"Май":70,
                    "Июн":40,"Июл":30,"Август":25,"Сентябр":40,
                    "Октябр":50,"Ноябр":45,"Декабр":35}

# === Регионы ===
regions = {
    "Хатлон": {"zone":"Водии","temp":(18,28)},
    "Суғд": {"zone":"Нисбатан кӯҳӣ","temp":(14,24)},
    "ВМКБ": {"zone":"Кӯҳӣ","temp":(0,9)},
    "НТҶ": {"zone":"Мухталиф","temp":(16,26)}
}

# === Доли по маҳсулотҳо ===
shares = {
    "Гандум":   {"Хатлон":0.40,"Суғд":0.35,"НТҶ":0.20,"ВМКБ":0.05},
    "Сабзавот": {"Хатлон":0.45,"Суғд":0.30,"НТҶ":0.20,"ВМКБ":0.05},
    "Картошка": {"Хатлон":0.25,"Суғд":0.40,"НТҶ":0.25,"ВМКБ":0.10},
    "Гӯшт":     {"Хатлон":0.38,"Суғд":0.32,"НТҶ":0.20,"ВМКБ":0.10},
    "Шир":      {"Хатлон":0.40,"Суғд":0.30,"НТҶ":0.20,"ВМКБ":0.10},
}

# === Базовые страновые ориентиры ===
base_country_targets = {
    "Гандум":   {"area": 230_000, "yield": 3.0},
    "Картошка": {"area": 50_000,  "yield": 20.5},
    "Сабзавот": {"area": 85_000,  "yield": 15.5},
    "Гӯшт":     {"volume": 360_000},
    "Шир":      {"volume": 1_150_000},
}

# === Функция естественной динамики (±10% по годам) ===
def evolve_value(prev_val):
    change = np.random.uniform(-0.1, 0.1)
    return prev_val * (1 + change)

# === Генерация показателей для каждого года ===
country_targets_by_year = {}
prev = base_country_targets
for y in years:
    country_targets_by_year[y] = {}
    for k, v in prev.items():
        if "area" in v:
            area = evolve_value(v["area"])
            yld = evolve_value(v["yield"])
            country_targets_by_year[y][k] = {"area": area, "yield": yld}
        else:
            vol = evolve_value(v["volume"])
            country_targets_by_year[y][k] = {"volume": vol}
    prev = country_targets_by_year[y]

# === Урожайные месяцы ===
harvest_months = {
    "Гандум": ["Июн","Июл"],
    "Картошка": ["Май","Июн","Сентябр"],
    "Сабзавот": ["Июн","Июл","Август","Сентябр"],
    "Гӯшт": months,
    "Шир": months
}

# === Базовые цены (сом/т) ===
base_prices = {
    "Гандум": (5000, 6000),
    "Картошка": (6000, 8000),
    "Сабзавот": (4000, 6000),
    "Гӯшт": (60000, 80000),
    "Шир": (8000, 10000),
}

# === Чувствительность цены к дефициту ===
price_alpha = {
    "Гандум": 0.25,
    "Картошка":0.20,
    "Сабзавот":0.20,
    "Гӯшт":0.15,
    "Шир":0.15,
}

# === Урожайность: фиксированные по регионам ===
yield_mult_fixed = {
    "Хатлон": 1.20,
    "Суғд": 1.05,
    "НТҶ": 0.75,
    "ВМКБ": 0.20
}

# === Торговля (импорт/экспорт bias) ===
trade_bias = {
    "Хатлон": {"imp": 0.5, "exp": 1.1},
    "Суғд": {"imp": 0.7, "exp": 1.0},
    "НТҶ": {"imp": 1.4, "exp": 0.5},
    "ВМКБ": {"imp": 1.1, "exp": 0.1}
}

# === Цены: региональный коэффициент ===
price_region_bias = {
    "Хатлон": 1.25,
    "Суғд": 1.10,
    "НТҶ": 0.49,
    "ВМКБ": 0.21
}

# === Базовый спрос (т/мес) и сезонный коэффициент ===
base_demand = {
    "Гандум": 30000,
    "Картошка":20000,
    "Сабзавот":25000,
    "Гӯшт":15000,
    "Шир":40000
}

season_factor = {
    "Январ": 0.9,"Феврал": 0.9,"Март": 1.0,"Апрел": 1.1,"Май": 1.2,
    "Июн": 1.3,"Июл": 1.3,"Август": 1.2,"Сентябр": 1.1,"Октябр": 1.0,
    "Ноябр": 0.95,"Декабр": 0.9
}

# === Региональные верхние границы самодостаточности (%) ===
region_selfsuff_cap = {
    "Хатлон": 60.0,
    "Суғд": 57.0,
    "НТҶ": 25.0,
    "ВМКБ": 5.0
}

rows = []

# --- Доп. параметры для реалистичности ---
ANOMALY_YEAR_PROB = 0.15      # вероятность аномального года (засуха/много осадков)
MAX_TEMP_DEVIATION = 6.0      # градусы, чтобы ограничить неправдоподобные значения
DEMAND_NOISE_SD = 0.12        # разброс спроса по регионам/месяцам
YIELD_YEAR_SD = 0.07          # годовой региональный эффект на урожайность
MONTHLY_YIELD_SD = 0.08       # месячный шум на урожайность
PRICE_JUMP_PROB = 0.03        # редкие рыночные шоки (скачки цен)

# --- Сгенерируем флаги аномальных лет и годовые региональные эффекты ---
anomaly_years = {y: (np.random.rand() < ANOMALY_YEAR_PROB) for y in years}
# годовые региональные мультипликаторы урожайности (малые вариации)
year_region_yield_effect = {
    y: {r: np.random.normal(1.0, YIELD_YEAR_SD) for r in regions} for y in years
}
# небольшой тренд спроса между регионами (чтобы не быть статичными)
region_demand_shift = {r: np.random.normal(1.0, 0.03) for r in regions}

# === Основной цикл ===
for year in years:
    ct = country_targets_by_year[year]

    # отражаем аномальный год: если засуха — уменьшаем осадки и урожай, если влажный — увеличим
    anomaly_multiplier_rain = 1.0
    anomaly_yield_multiplier = 1.0
    if anomaly_years[year]:
        # случайно решаем тип аномалии
        if np.random.rand() < 0.5:
            anomaly_multiplier_rain = np.random.uniform(0.6, 0.85)  # засуха
            anomaly_yield_multiplier = np.random.uniform(0.7, 0.95)
        else:
            anomaly_multiplier_rain = np.random.uniform(1.15, 1.5)  # влажный год
            anomaly_yield_multiplier = np.random.uniform(1.03, 1.2)

    wheat_area_country   = np.random.uniform(ct["Гандум"]["area"]*0.97, ct["Гандум"]["area"]*1.03)
    wheat_yield_country  = np.random.uniform(ct["Гандум"]["yield"]*0.97, ct["Гандум"]["yield"]*1.03)
    veg_area_country     = np.random.uniform(ct["Сабзавот"]["area"]*0.97, ct["Сабзавот"]["area"]*1.03)
    veg_yield_country    = np.random.uniform(ct["Сабзавот"]["yield"]*0.97, ct["Сабзавот"]["yield"]*1.03)
    pot_area_country     = np.random.uniform(ct["Картошка"]["area"]*0.97, ct["Картошка"]["area"]*1.03)
    pot_yield_country    = np.random.uniform(ct["Картошка"]["yield"]*0.97, ct["Картошка"]["yield"]*1.03)
    meat_country         = np.random.uniform(ct["Гӯшт"]["volume"]*0.97, ct["Гӯшт"]["volume"]*1.03)
    milk_country         = np.random.uniform(ct["Шир"]["volume"]*0.97, ct["Шир"]["volume"]*1.03)

    def with_noise(x, sd=0.05): return x * np.random.normal(1.0, sd)

    area_by_region = {
        "Гандум":   {r: with_noise(wheat_area_country * shares["Гандум"][r], sd=0.04) for r in regions},
        "Сабзавот": {r: with_noise(veg_area_country * shares["Сабзавот"][r], sd=0.04) for r in regions},
        "Картошка": {r: with_noise(pot_area_country * shares["Картошка"][r], sd=0.04) for r in regions},
    }

    vol_by_region = {
        "Гӯшт": {r: with_noise(meat_country * shares["Гӯшт"][r], sd=0.03) for r in regions},
        "Шир":  {r: with_noise(milk_country * shares["Шир"][r], sd=0.03) for r in regions},
    }

    for region, rinfo in regions.items():
        # небольшой региональный лаг/эффект на спрос
        regional_demand_factor = region_demand_shift[region]

        # === Растанипарварӣ ===
        for product in ["Гандум","Картошка","Сабзавот"]:
            annual_area = area_by_region[product][region]
            months_h = harvest_months[product]
            n_harvest = max(1, len(months_h))
            # регионно-годовой эффект урожайности
            region_year_yield_mul = year_region_yield_effect[year][region] * anomaly_yield_multiplier

            for month in months:
                # --- сила влияния месяца на урожай (случайный месячный шум) ---
                monthly_yield_noise = np.random.normal(1.0, MONTHLY_YIELD_SD)
                # иногда бывают локальные небольшие аномалии (редко)
                if np.random.rand() < 0.02:
                    monthly_yield_noise *= np.random.uniform(0.7, 1.25)

                is_harvest = month in months_h
                # косвенно распределяем площадь по месячнам уборки (как было), но добавляем случайные потери
                area = (annual_area / n_harvest) if is_harvest else 0.0
                # harvest loss — болезни, логистика, повреждения (рандом между 0.80 и 1.05)
                harvest_realization = np.random.uniform(0.80, 1.05) if is_harvest else 1.0

                base_yield_reg = {
                    "Гандум": wheat_yield_country,
                    "Картошка": pot_yield_country,
                    "Сабзавот": veg_yield_country
                }[product] * yield_mult_fixed[region] * region_year_yield_mul * monthly_yield_noise

                yield_month = base_yield_reg if is_harvest else 0.0
                # production теперь учитывает harvest_realization и небольшой независимый шум
                production = area * yield_month * harvest_realization * np.random.normal(1.0, 0.03)

                # спрос: базовый * сезонность * региональные сдвиги * малые месяц-регион шоки (чтобы уменьшить корреляцию)
                demand_shock = np.random.normal(1.0, DEMAND_NOISE_SD)
                # реальное потребление
                consumption = base_demand[product] * shares[product][region] \
                              * season_factor[month] * regional_demand_factor * demand_shock
                # корректируем consumption чтобы не уйти в нереальность (отсечение)
                consumption = max(0.1 * base_demand[product], consumption)

                # === Баланс и торговля (с теми же bias, но с рандомизацией) ===
                net_need = consumption - production
                if net_need > 0:
                    imp = net_need * np.random.uniform(0.55, 1.05) * trade_bias[region]["imp"]  # более вариативно
                    exp = 0.0
                else:
                    imp = 0.0
                    exp = (-net_need) * np.random.uniform(0.25, 0.95) * trade_bias[region]["exp"]

                eps = 1e-6
                self_suff = ((production + imp) / (consumption + eps)) * 100
                # Самодостаточность ограничиваем верхним cap (как было), но даём некоторую стохастику
                self_suff = min(region_selfsuff_cap[region] * np.random.uniform(0.95, 1.05), self_suff)
                self_suff = max(0.0, self_suff)

                # --- Цена: делаем реакцию на d/s ненадёжной (нелинейной) ---
                base_p = np.random.uniform(*base_prices[product])

                # относительное давление спроса/предложения
                d_s_ratio = consumption / (production + imp + eps)
                # мощность реакции: случайная между 1.0 и 1.4 -> нелинейный отклик
                price_power = np.random.uniform(1.0, 1.4)
                # базовая нелинейная реакция
                price_reaction = (d_s_ratio - 1.0)
                price = base_p * (1 + price_alpha[product] * (np.sign(price_reaction) * (abs(price_reaction) ** price_power)))
                # добавляем рынок-шоки (редко большой, обычно малый шум)
                if np.random.rand() < PRICE_JUMP_PROB:
                    # редкий скачок (например, локальная паника или сезонный ажиотаж)
                    price *= np.random.uniform(1.15, 1.45)
                else:
                    price += np.random.normal(0, base_p * 0.03)
                # региональный bias
                price = max(0.0, price * price_region_bias[region])

                # --- Климат: temp и rain с сохранением логики месячных диапазонов и ограничения ---
                # базовая температура + месячный сдвиг + шум
                temp = np.random.uniform(*rinfo["temp"]) + month_temp_shift[month] + np.random.normal(0, 1.6)
                # применяем аномалии по году на осадки
                rain = month_rain_shift[month] * anomaly_multiplier_rain + np.random.normal(0, 9.0)

                # Ограничиваем температуру в разумных пределах (чтобы не было 25° в январе в горах и т.п.)
                min_plausible = rinfo["temp"][0] + month_temp_shift[month] - MAX_TEMP_DEVIATION
                max_plausible = rinfo["temp"][1] + month_temp_shift[month] + MAX_TEMP_DEVIATION
                temp = float(np.clip(temp, min_plausible, max_plausible))

                rows.append([
                    year, month, region, product,
                    round(area,1), round(yield_month,2), round(production,1),
                    round(price,2), round(imp,1), round(exp,1),
                    round(consumption,1), round(self_suff,1),
                    round(temp,1), round(rain,1), rinfo["zone"]
                ])

        # === Чорводори (молоко, мясо) ===
        for product in ["Гӯшт","Шир"]:
            annual_prod = vol_by_region[product][region]
            # добавляем небольшой годовой вариативный фактор производства животных (сезонное размножение/забой)
            animal_prod_mul = np.random.normal(1.0, 0.04)
            for month in months:
                # производство равномерно по месяцам, но с сезонной и случайной вариацией
                seasonal_animal_mul = 1.0 + 0.05 * np.sin((months.index(month)/12.0)*2*np.pi + np.random.normal(0, 0.2))
                production = (annual_prod / 12.0) * animal_prod_mul * seasonal_animal_mul * np.random.normal(1.0, 0.03)

                demand_shock = np.random.normal(1.0, DEMAND_NOISE_SD)
                consumption = base_demand[product] * shares[product][region] \
                              * season_factor[month] * region_demand_shift[region] * demand_shock
                consumption = max(0.1 * base_demand[product], consumption)

                net_need = consumption - production
                if net_need > 0:
                    imp = net_need * np.random.uniform(0.55, 1.05) * trade_bias[region]["imp"]
                    exp = 0.0
                else:
                    imp = 0.0
                    exp = (-net_need) * np.random.uniform(0.25, 0.95) * trade_bias[region]["exp"]

                eps = 1e-6
                self_suff = ((production + imp) / (consumption + eps)) * 100
                self_suff = min(region_selfsuff_cap[region] * np.random.uniform(0.95, 1.05), self_suff)
                self_suff = max(0.0, self_suff)

                base_p = np.random.uniform(*base_prices[product])
                d_s_ratio = consumption / (production + imp + eps)
                price_power = np.random.uniform(1.0, 1.3)
                price_reaction = (d_s_ratio - 1.0)
                price = base_p * (1 + price_alpha[product] * (np.sign(price_reaction) * (abs(price_reaction) ** price_power)))
                if np.random.rand() < PRICE_JUMP_PROB:
                    price *= np.random.uniform(1.10, 1.35)
                else:
                    price += np.random.normal(0, base_p * 0.035)
                price = max(0.0, price * price_region_bias[region])

                temp = np.random.uniform(*rinfo["temp"]) + month_temp_shift[month] + np.random.normal(0, 1.6)
                rain = month_rain_shift[month] * anomaly_multiplier_rain + np.random.normal(0, 9.0)
                min_plausible = rinfo["temp"][0] + month_temp_shift[month] - MAX_TEMP_DEVIATION
                max_plausible = rinfo["temp"][1] + month_temp_shift[month] + MAX_TEMP_DEVIATION
                temp = float(np.clip(temp, min_plausible, max_plausible))

                rows.append([
                    year, month, region, product,
                    0.0, 0.0, round(production,1),
                    round(price,2), round(imp,1), round(exp,1),
                    round(consumption,1), round(self_suff,1),
                    round(temp,1), round(rain,1), rinfo["zone"]
                ])

# --- DataFrame и финальные колонки ---
df = pd.DataFrame(rows, columns=[
    "Сол","Моҳ","Минтақа","Маҳсулот",
    "Ҳаҷми замин (га)","Ҳосилнокӣ (т/га)","Истеҳсолот (т)",
    "Арзиш (сом/т)","Воридот (т)","Содирот (т)",
    "Истеъмолот (т)","Худтаъминкуни (%)",
    "Ҳарорат (°C)","Боришот (мм)","Минтақаи иқлимӣ"
])

# Переводим числовую самодостаточность в Бале/Не (как хотел)
df["Худтаъминкуни"] = np.where(df["Худтаъминкуни (%)"] >= 59, "Бале", "Не")

# Немного пост-обработки: убираем нулевые артефакты, ставим разумные типы
df["Арзиш (сом/т)"] = df["Арзиш (сом/т)"].round(2)
df["Истеҳсолот (т)"] = df["Истеҳсолот (т)"].round(1)
df["Истеъмолот (т)"] = df["Истеъмолот (т)"].round(1)
df["Воридот (т)"] = df["Воридот (т)"].round(1)
df["Содирот (т)"] = df["Содирот (т)"].round(1)

if __name__ == '__main__':
    display(df.head(20))

,Сол,Моҳ,Минтақа,Маҳсулот,Ҳаҷми замин (га),Ҳосилнокӣ (т/га),Истеҳсолот (т),Арзиш (сом/т),Воридот (т),Содирот (т),Истеъмолот (т),Худтаъминкуни (%),Ҳарорат (°C),Боришот (мм),Минтақаи иқлимӣ,Худтаъминкуни
0,2020,Январ,Хатлон,Гандум,0.0,0.00,0.0,11150.12,2322.3,0.0,7025.9,33.1,11.7,36.1,Водии,Не
1,2020,Феврал,Хатлон,Гандум,0.0,0.00,0.0,12912.67,2989.8,0.0,10385.9,28.8,16.7,34.2,Водии,Не
2,2020,Март,Хатлон,Гандум,0.0,0.00,0.0,10361.95,5475.4,0.0,13389.0,40.9,17.1,54.8,Водии,Не
3,2020,Апрел,Хатлон,Гандум,0.0,0.00,0.0,9841.28,4923.4,0.0,10996.2,44.8,22.4,51.5,Водии,Не
4,2020,Май,Хатлон,Гандум,0.0,0.00,0.0,8763.81,6597.4,0.0,13213.1,49.9,34.2,65.7,Водии,Не
5,2020,Июн,Хатлон,Гандум,41079.1,2.36,96079.1,5823.93,0.0,60490.4,18228.9,58.3,39.0,35.0,Водии,Не
6,2020,Июл,Хатлон,Гандум,41079.1,3.27,126656.7,5582.48,0.0,116097.5,13304.4,59.4,36.6,48.0,Водии,Бале
7,2020,Август,Хатлон,Гандум,0.0,0.00,0.0,12645.10,4411.7,0.0,13817.7,31.9,30.0,26.2,Водии,Не
8,2020,Сентябр,Хатлон,Гандум,0.0,0.00,0.0,12549.07,3785.5,0.0,11927.5,31.7,27.1,21.0,Водии,Не
9,2020,Октябр,Хатлон,Гандум,0.0,0.00,0.0,11783.11,3532.3,0.0,10138.6,34.8,27.4,39.2,Водии,Не


In [19]:
df.drop(columns=['Боришот (мм)'], inplace=True)

In [21]:
# === Сохранение ===
df.to_csv("taj_agriculture_2020_2024_tj.csv", index=False, encoding="utf-8-sig")
print("✅ Синтетический датасет успешно создан и сохранён как 'tajikistan_agriculture_2023_2024_tj.csv'")

✅ Синтетический датасет успешно создан и сохранён как 'tajikistan_agriculture_2023_2024_tj.csv'
